In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score, matthews_corrcoef, average_precision_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn.metrics import roc_curve
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from scipy.stats import randint, uniform
from pandarallel import pandarallel
from sklearn.model_selection import StratifiedKFold

import math
import sys
sys.path.append('/home/ss2686/03_DICTrank')

import argparse
from scripts.evaluation_functions import evaluate_classifier, optimize_threshold_j_statistic

# Initialize pandarallel for parallel processing
pandarallel.initialize()
import gzip

data_path = '../data/processed_binarised__splits/'

# Define the path to your gzip-compressed image_features.csv.gz file
csv_file_path = '../data/LINCSL1000/LINCSL1000_processed.csv.gz'


def create_molecule_dict(csv_file_path):
    molecule_dict = {}

    with gzip.open(csv_file_path, 'rt') as f:
        next(f)  # Skip the first line (header)
        for line in f:
            data = line.strip().split(',')
            smiles = data[0]
            features = np.array(data[1:979], dtype=float)
            molecule_dict[smiles] = features
    
    return molecule_dict

# Assuming you call create_molecule_dict once to create the dictionary
molecule_dict = create_molecule_dict(csv_file_path)

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

def calculate_tanimoto_similarity(molecule1, molecule2):
    molecule1_fp =  AllChem.GetMorganFingerprintAsBitVect(molecule1, 2, nBits=2048)
    molecule2_fp =  AllChem.GetMorganFingerprintAsBitVect(molecule2, 2, nBits=2048)
    return DataStructs.TanimotoSimilarity(molecule1_fp, molecule2_fp)

def fetch_similar_profiles(smiles, exclude_smiles_set=None):
    if exclude_smiles_set is None:
        exclude_smiles_set = set()

    query_mol = Chem.MolFromSmiles(smiles)
    similar_profiles = []

    for key_smiles in molecule_dict:
        if key_smiles in exclude_smiles_set:
            continue

        key_mol = Chem.MolFromSmiles(key_smiles)
        similarity = calculate_tanimoto_similarity(query_mol, key_mol)
        if similarity > 0.7:
            similar_profiles.append(molecule_dict[key_smiles])

    if len(similar_profiles) > 0:
        return np.mean(similar_profiles, axis=0)
    else:
        return None

# Modify the generate_cellpainting function
def generate_lincs(smiles, behave="Train", exclude_smiles_set=None):
    profile = molecule_dict.get(smiles)
    if profile is not None:
        return profile
    elif(behave=="Train"):
        return fetch_similar_profiles(smiles, exclude_smiles_set)
    else:
        return molecule_dict.get(smiles, np.zeros(978, dtype=float))
    
results = {}
held_out_results = []

for dataset in os.listdir(data_path):
    
    # Exclude hidden files or directories like .ipynb_checkpoints
    if dataset.startswith('.'):
        continue
    print(dataset)


    # Get all the file names for this dataset
    all_files = os.listdir(os.path.join(data_path, dataset))

    # Extract activity names by removing the _train.csv.gz or _test.csv.gz from file names
    activity_names = list(set([f.replace("_train.csv.gz", "").replace("_test.csv.gz", "")  for f in all_files if not f.startswith(".ipynb_checkpoints")]))

    for activity in tqdm(activity_names, desc="Processing activities"):
        
        train_path = os.path.join(data_path, dataset, f"{activity}_train.csv.gz")
        test_path = os.path.join(data_path, dataset, f"{activity}_test.csv.gz")

        train_df = pd.read_csv(train_path, compression='gzip')#.sample(20)
        test_df = pd.read_csv(test_path, compression='gzip')#.sample(20)

        train_smiles_set = set(train_df['Standardized_SMILES'].tolist())
        test_smiles_set = set(test_df['Standardized_SMILES'].tolist())
        
        X_train = train_df['Standardized_SMILES'].parallel_apply(lambda x: generate_lincs(x, "Train", test_smiles_set))
        X_train = np.array(X_train.to_list())
        
        X_test = test_df['Standardized_SMILES'].parallel_apply(lambda x: generate_lincs(x, "Test"))
        X_test = np.array(X_test.to_list())
        
        y_train = train_df[activity]
        y_test = test_df[activity]

        failed_train_indices = [i for i, lincs in enumerate(X_train) if lincs is None]
        failed_test_indices = [i for i, lincs in enumerate(X_test) if lincs is None]
        
        # Drop those indices from X_train, X_test, y_train, and y_test
        X_train = np.delete(X_train, failed_train_indices, axis=0)
        X_train = np.vstack(X_train)
        y_train = y_train.drop(failed_train_indices).reset_index(drop=True)

        X_test = np.delete(X_test, failed_test_indices, axis=0)
        X_test = np.vstack(X_test)
        y_test = y_test.drop(failed_test_indices).reset_index(drop=True)

        # If you want to drop the rows from train_df and test_df as well
        train_df = train_df.drop(failed_train_indices).reset_index(drop=True)
        test_df = test_df.drop(failed_test_indices).reset_index(drop=True)
        
        print(train_df[activity].value_counts())
        print(test_df[activity].value_counts())
        
        # Classification
        model = RandomForestClassifier(n_jobs=40)
            
        # Hyperparameter Optimization
        param_dist_classification = {'max_depth': randint(10, 20),
                          'max_features': randint(40, 50),
                          'min_samples_leaf': randint(5, 15),
                          'min_samples_split': randint(5, 15),
                          'n_estimators':[200, 300, 400, 500, 600],
                          'bootstrap': [True, False],
                          'oob_score': [False],
                          'random_state': [42],
                          'criterion': ['gini', 'entropy'],
                          'n_jobs': [40],
                          'class_weight' : [None, 'balanced']
                         }
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)   
            
        classification_search = HalvingRandomSearchCV(
                model,
                param_dist_classification,
                factor=3,
                cv=inner_cv,
                random_state=42,
                verbose=1,
                n_jobs=40)
            
        classification_search.fit(X_train, y_train)
        best_model = classification_search.best_estimator_
            
        # Random Over-sampling and Threshold Optimization
        sampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
            
        pipeline = Pipeline(steps=[('sampler', sampler), ('model', best_model)])
        pipeline.fit(X_train, y_train)
            
        # Predict using threshold-optimized model
        predictions_train = pipeline.predict(X_train)
        probs_train = pipeline.predict_proba(X_train)[:, 1]
        probs_test = pipeline.predict_proba(X_test)[:, 1]
            
        # Use the optimize_threshold_j_statistic function to find the best threshold
        best_threshold = optimize_threshold_j_statistic(y_train, probs_train)
        #Apply the best threshold to get binary predictions on the test data
        predictions_test = (probs_test >= best_threshold).astype(int)
            
        # Calculate CV AUC using threshold-optimized model
        cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, n_jobs=-1, scoring='roc_auc')

        results[activity] = {
                'CV_AUC_mean': np.mean(cv_scores),
                'CV_AUC_std': np.std(cv_scores),
                **evaluate_classifier(y_test, predictions_test, probs_test)
            }
   
        
        held_out_data = {
            'Dataset': dataset,
            "Actviity": activity,
            'SMILES': test_df['Standardized_SMILES'],
            'True_Value': y_test,
            'Prediction': predictions_test,
            'Probability': probs_test,
            'Best_Threshold': best_threshold
        }
        
        held_out_results.append(pd.DataFrame(held_out_data))  

        #Save results at each step
        pd.DataFrame(results).T.to_csv('./LINCSL1000_model_results.csv')
            
        

# Save results
results_df = pd.DataFrame(results).T.reset_index(drop=False)
results_df = results_df.rename(columns={'index': 'endpoint'})
results_df.to_csv('./LINCSL1000_model_results.csv', index=False)

# Concatenate and save held-out test set results
pd.concat(held_out_results).to_csv('./LINCSL1000_model_held_out_test_results.csv', index=False)

INFO: Pandarallel will run on 76 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
cardiotox_with_sider_inactives


Processing activities:   0%|                              | 0/1 [00:00<?, ?it/s]/tmp/ipykernel_3094968/1983872822.py:119: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(X_train.to_list())


1    416
0    223
Name: Cardiotox (with SIDER inactives), dtype: int64
1    65
0    25
Name: Cardiotox (with SIDER inactives), dtype: int64
n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 20
max_resources_: 639
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 31
n_resources: 20
Fitting 5 folds for each of 31 candidates, totalling 155 fits
----------
iter: 1
n_candidates: 11
n_resources: 60
Fitting 5 folds for each of 11 candidates, totalling 55 fits
----------
iter: 2
n_candidates: 4
n_resources: 180
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 3
n_candidates: 2
n_resources: 540
Fitting 5 folds for each of 2 candidates, totalling 10 fits


Processing activities: 100%|██████████████████████| 1/1 [01:28<00:00, 88.25s/it]


cardiotox_with_sider_actives


...

Processing activities: 100%|██████████████████████| 1/1 [01:18<00:00, 78.26s/it]


DICTrank


Processing activities:   0%|                              | 0/1 [00:00<?, ?it/s]/tmp/ipykernel_3094968/1983872822.py:119: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(X_train.to_list())


1    416
0     90
Name: DICTrank, dtype: int64
1    65
0    25
Name: DICTrank, dtype: int64
n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 20
max_resources_: 506
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 25
n_resources: 20
Fitting 5 folds for each of 25 candidates, totalling 125 fits
----------
iter: 1
n_candidates: 9
n_resources: 60
Fitting 5 folds for each of 9 candidates, totalling 45 fits
----------
iter: 2
n_candidates: 3
n_resources: 180
Fitting 5 folds for each of 3 candidates, totalling 15 fits


Processing activities: 100%|██████████████████████| 1/1 [01:28<00:00, 88.11s/it]


In [3]:
df = pd.read_csv('./LINCSL1000_model_held_out_test_results.csv')
df

,Dataset,Actviity,SMILES,True_Value,Prediction,Probability,Best_Threshold
0,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),O=C(C1CCCCC1)[NH+]1CC(=O)[NH+]2CCc3ccccc3C2C1,1,1,0.619192,0.604273
1,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),CC(C(=O)[O-])c1ccc(-c2ccccc2)c(F)c1,1,0,0.600085,0.604273
2,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),C[NH+](C)CCC=C1c2ccccc2CCc2ccccc21,1,0,0.602112,0.604273
3,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),CC(=O)[NH+]1CCN(c2ccc(OCC3COC(Cn4ccnc4)(c4ccc(...,1,1,0.618311,0.604273
4,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),Cc1nccn1CC1CCc2c(c3ccccc3n2C)C1=O,1,1,0.606801,0.604273
...,...,...,...,...,...,...,...
488,DICTrank,DICTrank,CCC1(c2ccccc2)C(=O)NCNC1=O,0,0,0.809186,0.879318
489,DICTrank,DICTrank,CCOC(=O)[NH+]1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1,0,1,0.887670,0.879318
490,DICTrank,DICTrank,CCCSc1ccc2[n-]c(=NC(=O)OC)[n-]c2c1,0,0,0.862238,0.879318
491,DICTrank,DICTrank,CCC[NH+](CCC)S(=O)(=O)c1ccc(C(=O)[O-])cc1,0,0,0.790788,0.879318


In [4]:
test_df

,Standardized_SMILES,Standardized_InChI,DICTrank
0,O=c1n(CCC[NH+]2CCN(c3cccc(Cl)c3)CC2)nc2ccccn12,InChI=1S/C19H22ClN5O/c20-16-5-3-6-17(15-16)23-...,1
1,CC(C(=O)[O-])c1ccc(-c2ccccc2)c(F)c1,InChI=1S/C15H13FO2/c1-10(15(17)18)12-7-8-13(14...,1
2,C[NH+](C)CCC=C1c2ccccc2CCc2ccccc21,InChI=1S/C20H23N/c1-21(2)15-7-12-20-18-10-5-3-...,1
3,CC(=O)[NH+]1CCN(c2ccc(OCC3COC(Cn4ccnc4)(c4ccc(...,InChI=1S/C26H28Cl2N4O4/c1-19(33)31-10-12-32(13...,1
4,Cc1nccn1CC1CCc2c(c3ccccc3n2C)C1=O,InChI=1S/C18H19N3O/c1-12-19-9-10-21(12)11-13-7...,1
...,...,...,...
85,CCC1(c2ccccc2)C(=O)NCNC1=O,InChI=1S/C12H14N2O2/c1-2-12(9-6-4-3-5-7-9)10(1...,0
86,CCOC(=O)[NH+]1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1,InChI=1S/C22H23ClN2O2/c1-2-27-22(26)25-12-9-15...,0
87,CCCSc1ccc2[n-]c(=NC(=O)OC)[n-]c2c1,InChI=1S/C12H14N3O2S/c1-3-6-18-8-4-5-9-10(7-8)...,0
88,CCC[NH+](CCC)S(=O)(=O)c1ccc(C(=O)[O-])cc1,"InChI=1S/C13H19NO4S/c1-3-9-14(10-4-2)19(17,18)...",0


In [5]:
y_test

0     1
1     1
2     1
3     1
4     1
     ..
85    0
86    0
87    0
88    0
89    0
Name: DICTrank, Length: 90, dtype: int64